Восстанавливаем указанную ассесорами релевантность по медиане и дисперсии

In [1]:
import numpy as np
import pandas as pd

In [3]:
# basic preprocessing
col_types = {u'id': np.int, 
             u'query': np.str, 
             u'product_title': np.str,
             u'product_description': np.str, 
             u'median_relevance': np.int, 
             u'relevance_variance': np.float}
df = pd.read_csv('../kaggle/clean_train.csv', dtype=col_types, index_col=u'id')
df = df.drop (["query","product_title","product_description"], axis=1)

Перебираем все возможные тройки, четверки и пятерки оценок, считаем их медиану и дисперсию

In [10]:
import math
def median (arr):
    l = len (arr)
    med_id = l / 2
    return sorted(arr)[med_id]

def variance (arr):
    mean = 1. *sum (arr) / len(arr)
    var = 1. * sum (map (lambda x: (x-mean)**2, arr)) / len(arr)
    return round(math.sqrt(var)*1000)/1000.0

In [11]:
from collections import defaultdict
med_var_2_relevance = defaultdict (list)

for k,v in sorted (map (lambda x: ((median(x), variance(x)), x),
        [(i,j,k)
         for i in range (1,5)     
         for j in range (i,5)         
         for k in range (j,5)
        ] +
        [(i,j,k,l)
         for i in range (1,5)     
         for j in range (i,5)         
         for k in range (j,5)
         for l in range (k,5)
        ]+
        [(i,j,k,l,m)
         for i in range (1,5)     
         for j in range (i,5)         
         for k in range (j,5)
         for l in range (k,5)
         for m in range (l,5)
        ]
    )):
    med_var_2_relevance[k].append (v)

Находим оценки, которые могли составить указанные медиану и вариацию

In [12]:
from itertools import groupby
from itertools import chain

def var2marks(mark_and_var):  
    mark, var = (mark_and_var[0],mark_and_var[1])
    var2marksList = {}
    
    near_values = filter (lambda x: (x[0][0] == mark and abs(x[0][1]- var)<0.002), med_var_2_relevance.items())
    if (len(near_values) > 0):        
        return [(key, 1.*len(list(group))/len(list(chain(*near_values[0][1])))) 
                for key, group in groupby(sorted(chain(*near_values[0][1])))]
    else:
        return np.nan

Сохраняем списки оценок в датафрейм

In [13]:
df["relevance_list"] = df[["median_relevance", "relevance_variance"]].apply (var2marks, axis=1)

Следующее шаманство разворачивает список в новый, более длинный датафреймь

In [5]:
def hasI (df, i):
    return df["relevance_list"].apply (lambda x: type(x) == list and len(filter (lambda y: y[0]==i,x)) > 0)

def getI (df, i):
    return df["relevance_list"].apply (lambda x:  filter (lambda y: y[0]==i,x)[0] 
                                       if type(x) == list and
                                       len(filter (lambda y: y[0]==i,x)) > 0
                                       else  (np.nan,np.nan))\
                               .apply (lambda s: pd.Series ({"relevance":s[0], "weight":s[1]}))

In [10]:
new_df = None
for i in range(1,5):
    current_df = df.copy()
    current_df = current_df[hasI(current_df, i)]
    current_df [["relevance","weight"]] = getI(df, i) 
    
    if new_df is None:
        new_df = current_df
    else:
        new_df = pd.concat ([new_df, current_df])

In [11]:
new_df1 = new_df.reset_index().drop (["median_relevance","relevance_variance", "relevance_list"],axis=1)
new_df1.to_csv ("restored_relevance_train.csv")

In [15]:
new_df1

,id,relevance,weight
0,1,1,1.000000
1,7,1,0.166667
2,8,1,0.200000
3,10,1,1.000000
4,17,1,0.200000
5,64,1,0.333333
6,65,1,0.333333
7,66,1,0.166667
8,72,1,0.200000
9,80,1,0.200000
